## IMPORTING NECESSARY LIBRARIES

In [66]:
import os
import cv2
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline

## DEFINING PATH FOR DATASET

In [67]:
binary_0 = r'C:\Data\SRI\Chest XRay TB\Normal'
binary_1 = r'C:\Data\SRI\Chest XRay TB\Tuberculosis'

## GETTING TRAIN/VALIDATION/TEST DATA

In [68]:
X = []
Y = []

In [69]:
binary_0_list = os.listdir(binary_0)

for row in binary_0_list:
    image_path1 = os.path.join(binary_0, row)

    image_read = cv2.imread(image_path1, 1) 
    
    image_final = cv2.resize(image_read, (128, 128))        
        
    X.append(image_final)
    Y.append(0)

In [70]:
binary_1_list = os.listdir(binary_1)

for row in binary_1_list:
    image_path1 = os.path.join(binary_1, row)

    image_read = cv2.imread(image_path1, 1) 
    
    image_final = cv2.resize(image_read, (128, 128))        
        
    X.append(image_final)
    Y.append(1)

In [71]:
X = np.array(X)
Y = np.array(Y)

X = X/255

## TRAIN/VALIDATION/TEST SPLIT

In [72]:
X_train_temp, X_test, Y_train_temp, Y_test = train_test_split(X, Y, test_size=0.2)                   

In [73]:
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train_temp, Y_train_temp, test_size=0.25)

## CONVOLUTION BLOCKS

In [74]:
#down sampling
def down_samp(x, filters):
    c = keras.layers.Conv2D(filters, (3, 3), padding="same", strides=1, activation="relu")(x)
    c = keras.layers.Conv2D(filters, (3, 3), padding="same", strides=1, activation="relu")(c)
    p = keras.layers.MaxPooling2D((2, 2), (2, 2))(c)
    
    return p
    
#bottleneck 
def bottlenock(x, filters):
    c = keras.layers.Conv2D(filters, (3, 3), padding="same", strides=1, activation="relu")(x)
    c = keras.layers.Conv2D(filters, (3, 3), padding="same", strides=1, activation="relu")(c)
    
    return c

## CNN ARCHITECTURE

In [75]:
def unet():
    f = [8, 16, 32, 64, 128]
    
    inputs = keras.layers.Input((128, 128, 3))
    
    p0 = inputs
    p1 = down_samp(p0, f[0])
    p2 = down_samp(p1, f[1])
    p3 = down_samp(p2, f[2])
    p4 = down_samp(p3, f[3])
    
    bn = bottlenock(p4, f[4])
    
    inp = keras.layers.Flatten()(bn)
    middle = keras.layers.Dense(5462, activation='relu')(inp) 
    outputs = keras.layers.Dense(1, activation='sigmoid')(middle)
    
    model = keras.models.Model(inputs, outputs)
    
    return model

In [76]:
model = unet()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

In [77]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 128, 128, 8)       224       
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 128, 128, 8)       584       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 64, 64, 8)         0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 64, 64, 16)        1168      
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 64, 64, 16)        2320      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 32, 32, 16)        0   

## TRAINING THE UNET MODEL

In [78]:
model.fit(
    X_train,
    Y_train,
    epochs=5,
    validation_data=(X_valid, Y_valid),
)

Epoch 1/5
79/79 [==============================] - 74s 875ms/step - loss: 0.3444 - accuracy: 0.8778 - val_loss: 0.1928 - val_accuracy: 0.9202
Epoch 2/5
79/79 [==============================] - 69s 869ms/step - loss: 0.1828 - accuracy: 0.9353 - val_loss: 0.1502 - val_accuracy: 0.9381
Epoch 3/5
79/79 [==============================] - 70s 885ms/step - loss: 0.1769 - accuracy: 0.9437 - val_loss: 0.1462 - val_accuracy: 0.9476
Epoch 4/5
79/79 [==============================] - 71s 901ms/step - loss: 0.1523 - accuracy: 0.9433 - val_loss: 0.1521 - val_accuracy: 0.9369
Epoch 5/5
79/79 [==============================] - 74s 933ms/step - loss: 0.1012 - accuracy: 0.9623 - val_loss: 0.1099 - val_accuracy: 0.9643


## EVALUATION/PREDICTION ON TEST DATA

In [79]:
evalu = model.evaluate(X_test, Y_test, verbose=0)
print('The accuracy of model on Test Data-Set is = '"{:.2f}".format(evalu[1]*100), '%')

The accuracy of model on Test Data-Set is = 97.02 %


In [80]:
result_temp = model.predict(X_test)
result_temp = result_temp < 0.5

In [81]:
results = []
for i in result_temp:
    if i:
        results.append(0)
    else:
        results.append(1)

In [82]:
print('True', ' ','Predicted')
for i in range(result.shape[0]):
    print(Y_test[i], '    ',results[i])

True   Predicted
0      0
0      0
1      1
0      0
0      0
1      1
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
1      1
0      0
0      0
1      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
1      1
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
1      1
0      0
0      0
1      1
0      0
1      1
0      0
0      0
0      0
1      1
1      1
1      1
1      1
0      0
0      0
0      0
0      0
0      0
1      1
0      0
0      0
1      1
1      1
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
1      1
0      0
1      1
0      0
0      0
0      0
0      0
1      1
0      1
1      1
1      1
0      0
0      0
0      1
0      0
0      0
0      0
0      0
0      1
0      0
0      0
0      0
1      1
0      0
0      0
0      0
0      0
0      0
0      0
0 